In [1]:
# pip install pyyaml

In [2]:
import pandas as pd
import yaml

with open('settings.yml', 'r') as file:
    settings = yaml.safe_load(file)

df_final_demo = pd.read_csv(settings['data']['src']['df_final_demo'])
df_pt1 = pd.read_csv(settings['data']['src']['pt_1'], parse_dates=['date_time'])
df_pt2 = pd.read_csv(settings['data']['src']['pt_2'], parse_dates=['date_time'])
df_final_experiement_clients = pd.read_csv(settings['data']['src']['df_final_experiment_clients'])

df_final_web_data = pd.concat([df_pt1,df_pt2])

In [3]:
# web_data_summary

# target df:

# client_id - id of client
# confirmed - has reached the confirmed step
# avg_step_duration - average time spent on each step
# error_count - number of teams they went back to a previous step
# had_error - use went back a step at least 1 time
# no_of_visits
# no_of_confirmed

In [4]:
clients_who_confirmed = df_final_web_data[df_final_web_data['process_step']=='confirm']
clients_who_confirmed = clients_who_confirmed.drop_duplicates(subset='client_id')

In [5]:
df_final_web_data = df_final_web_data.drop_duplicates()
df_final_web_data = df_final_web_data.sort_values(by=['visit_id','date_time'])
df_final_web_data = df_final_web_data.reset_index(drop=True)
df_final_web_data['next_date_time'] = (df_final_web_data.sort_values(by='date_time').groupby('visit_id')['date_time'].shift(-1))
df_final_web_data['duration'] = df_final_web_data['next_date_time'] - df_final_web_data['date_time']

In [6]:
df_avg_step_durations = df_final_web_data.groupby(by='client_id')[['duration']].mean()

In [7]:
# how are we going to define an error 
# - is hitting the same page twice an error
# - is it only an error if you go back a step?
# - is it only an error if you go back a step within the same visit?

# lets review the "documentation"
# Error Rates: If there’s a step where users go back to a previous step,
# it may indicate confusion or an error. You should consider moving from 
# a later step to an earlier one as an error

In [8]:
df_final_web_data = df_final_web_data.sort_values(['client_id','date_time'])
process_map = {'start':0, 'step_1':1, 'step_2':2, 'step_3':3, 'confirm':4}
df_final_web_data['process_step_id'] = df_final_web_data['process_step'].map(process_map)
df_final_web_data['previous_process_step_id'] = (df_final_web_data.sort_values(by=['client_id','date_time']).groupby('client_id')['process_step_id'].shift(1))
df_final_web_data['is_error'] = df_final_web_data['process_step_id'] < df_final_web_data['previous_process_step_id']
df_final_web_data[df_final_web_data['client_id'].isin(df_final_web_data['client_id'].sample(5))][['client_id','date_time','process_step_id','previous_process_step_id','is_error']]
df_client_error = df_final_web_data[df_final_web_data['is_error']==True].groupby('client_id')['is_error'].agg([max,len])

In [9]:
df_client_visits = df_final_web_data.groupby('client_id')[['visit_id']].nunique()

In [10]:
df_client_confirms = df_final_web_data[['client_id','process_step']][df_final_web_data['process_step']=='confirm'].groupby('client_id').count()
df_client_confirms.columns = ['no_of_confirms']

In [11]:
clients_who_confirmed['is_confirmed'] = 1
clients_who_confirmed = clients_who_confirmed[['client_id', 'is_confirmed']]
df_avg_step_durations = df_avg_step_durations.reset_index()
df_client_error = df_client_error.reset_index()
df_client_error.columns=['client_id','had_error','error_count']
df_client_error
df_client_visits = df_client_visits.reset_index()
df_client_visits.columns=['client_id', 'no_of_visits']
df_client_confirms = df_client_confirms.reset_index()

In [12]:
dfs =[df_client_visits, clients_who_confirmed, df_avg_step_durations, df_client_error, df_client_confirms]
dfs = [df.set_index('client_id') for df in dfs]

web_data_summary = dfs[0].join(dfs[1:])

In [13]:
web_data_summary['had_error'] = web_data_summary['had_error'].fillna(False)
web_data_summary = web_data_summary.fillna(0)
web_data_summary['is_confirmed'] = web_data_summary['is_confirmed'].map({1:True, 0:False})
web_data_summary[['error_count','no_of_confirms']] = web_data_summary[['error_count','no_of_confirms']].astype(int)

In [14]:
web_data_summary

,no_of_visits,is_confirmed,duration,had_error,error_count,no_of_confirms
client_id,,,,,,
169,1,True,0 days 00:00:53.250000,False,0,1
336,1,False,0 days 00:15:48,False,0,0
546,1,True,0 days 00:00:33.250000,False,0,1
555,1,True,0 days 00:00:39.500000,False,0,1
647,1,True,0 days 00:01:34.250000,False,0,1
...,...,...,...,...,...,...
9999729,3,True,0 days 00:01:07.500000,True,2,1
9999768,1,True,0 days 00:00:44.181818181,True,3,1
9999832,1,False,0 days 00:00:08,False,0,0
